In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import mojimoji
import re
import jaconv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import warnings
from sklearn import preprocessing
import MeCab
def parse_text(text, debug=False):
    text = re.sub(r'［[^］]+］', ' ', text)  
#     text = re.sub(r'（[^）]+）', ' ', text)  
    text = re.sub(r'○', ' ', text)    
    text = re.sub(r'×', '', text)
    text = re.sub(r'※', '', text)    
#     text = re.sub(r'｜', ' ', text)
    text = re.sub(r'[\s、]', '' , text)
#     text = re.sub(r'一', ' ', text)
#     text = re.sub(r'…', ' ', text)  
#     text = re.sub(r'―', ' ', text)
    text = re.sub(r'[0-9]', '0', text)
    return text

def tokenize1(text):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    l = []
    while node:
        l.append(node.surface)
        node = node.next
    return l
traindf=pd.read_csv("data/train.csv")
testdf=pd.read_csv("data/test.csv")
df=pd.concat([traindf,testdf])
df.body=df.body.map(parse_text)
traindf.body=traindf.body.map(parse_text)
testdf.body=testdf.body.map(parse_text)
y = df['author'].values
mecab = MeCab.Tagger('-Owakati')
count = CountVectorizer(analyzer=tokenize1)
bags = count.fit_transform(df.body.values)

features = count.get_feature_names()

bodyvec = pd.DataFrame(bags.toarray(), columns=features)
wordbody=bodyvec.sum().sort_values(ascending=False)
bodyvec1 = bodyvec[list(wordbody[wordbody > 37].index)]
xsum = bodyvec1.sum(axis=1)

xsum = np.array(xsum).reshape(len(xsum), 1)
newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec1/xsum*100)], axis=1)
train=newdf.dropna().drop(["writing_id","body",],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

mmscaler = preprocessing.MinMaxScaler()

mmscaler.fit(pd.concat([X,test]))
X = mmscaler.transform(X)
test= mmscaler.transform(test)


In [4]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 0) # 80%のデータを学習データに、20%を検証データにする
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(train_X,train_y)
pred = np.where(mlpr.predict(test_X)>0.30,1,0)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[919   0]
 [  2  73]]
accuracy =  0.9979879275653923
precision =  1.0
recall =  0.9733333333333334
f1 score =  0.9864864864864865


In [9]:
pred = np.where(mlpr.predict(test_X)>0.3,1,0)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[919   0]
 [  2  73]]
accuracy =  0.9979879275653923
precision =  1.0
recall =  0.9733333333333334
f1 score =  0.9864864864864865


In [10]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(X,y)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(128, 128, 128), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=42, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [19]:
pred = np.where(mlpr.predict(test)>0.33,1,0)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index = False)

In [15]:
pred=mlpr.predict(test)

In [16]:
data=pd.concat([pd.concat([pd.DataFrame(X),pd.DataFrame(test)]),pd.concat([y,pd.Series(pred)])],axis=1)
data.columns=list(wordbody[wordbody>37].index)+["target"]
data

,の,。,に,は,た,て,を,が,で,と,...,シャレ,なぜだか,系譜,ベズィメンスキー,輿,めまい,菜の花,駭,万象,target
0,0.467967,0.332256,0.591386,0.382519,0.170613,0.199220,0.506616,0.214485,0.459610,0.103064,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.412500,0.213000,0.530769,0.480635,0.260313,0.238400,0.378906,0.371250,0.302500,0.292917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.377622,0.238322,0.222700,0.336109,0.513986,0.666853,0.466346,0.192308,0.384615,0.172494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.381737,0.363503,0.456875,0.436848,0.605165,0.285509,0.549074,0.284057,0.115269,0.180015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
4,0.474085,0.429242,0.546352,0.487778,0.697082,0.419079,0.408708,0.291047,0.187387,0.254802,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0.445693,0.218052,0.424085,0.317166,0.367041,0.327391,0.386002,0.350187,0.281523,0.377237,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006545
1416,0.294352,0.231509,0.312072,0.427905,0.343086,0.222194,0.336212,0.229528,0.131596,0.209311,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000137
1417,0.129730,0.230270,0.459044,0.593835,0.264865,0.128865,0.262162,0.178378,0.297297,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030456
1418,0.189335,0.237226,0.274073,0.317473,0.341082,0.181033,0.220975,0.133648,0.120655,0.226853,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004802


In [17]:
data[(data.target<0.7)&(data.target>0.29)].target
# 85 0
#193 1
#277 0
#291 0 0.43✖︎
#412 1 0.40✖︎
#547 1 0.41
#625 1 0.58
#682 0 0.43
#702 1 
#774 1
#815 1 0.38


85      0.365065
193     0.670269
277     0.355229
291     0.439504
412     0.401968
547     0.418893
625     0.581322
682     0.430803
702     0.589994
774     0.487702
815     0.383599
955     0.648227
1029    0.591053
1112    0.375417
1116    0.604742
1140    0.335525
1200    0.410798
1281    0.329586
Name: target, dtype: float64

In [18]:
testdf.ix[data[(data.target<0.7)&(data.target>0.29)].index]

,writing_id,body
85,277,虫の中でも人間に評判のよくないものの随一《ずいいち》は蛆《うじ》である。「蛆虫めら」というの...
193,674,江口は決して所謂快男児ではない。もっと複雑なもっと陰影に富んだ性格の所有者だ。愛憎の動き方な...
277,948,どうもみんな佳《よ》い言葉を使い過ぎます。美辞を姦するおもむきがあります。鴎外《おうがい》が...
291,998,一私が生れて始めて蓄音器と云うものを見聞いたのはもう十四五年前父が英国から土産に買って来たも...
412,1379,昔々バグダツドのマホメツト教のお寺の前に一人の乞食が寝て居りました。丁度その時説教がすんだの...
547,1862,或る一つの作品を書かうと思つてそれが色々の径路を辿《たど》つてから出来上がる場合と直ぐ初めの...
625,2137,菊池は生き方が何時も徹底している。中途半端のところにこだわっていない。彼自身の正しいと思うと...
682,2321,十年前は阿佐ヶ谷に住んで居りやはり目下と同様吶々と小説ばかり書いて居りました。もう十年も経つ...
702,2385,すべて背景を用いない。宦官《かんがん》が二人話しながら出て来る。――今月も生み月になっている...
774,2649,ナポレオンの遺書――セント・ヘレナの島で臨終より三週間ほど前に彼が自ら口述し浄書したもので現...


In [79]:
data[(data.target<0.7)&(data.target>0.29)].target
#85 0 0.43
#412 1 0.55
#547 1 0.54
#625 1 0.0.47
#682 0 9,31 
#774 1 0.59
#825 0 0.30
#955 1 0.61
#962 0 0.37
#1029 1 0.51
#1082 0 0.31
#1112 0 0.44
#1116 1 0.6
#1140 1 0.52
#1194 1 0.68
#1281 0 0.46
#1344 0 0.36

19      0.541001
88      0.498581
96      0.356384
97      0.686285
130     0.617800
          ...   
1359    0.679964
1361    0.444833
1363    0.644160
1400    0.502069
1406    0.634550
Name: target, Length: 70, dtype: float64

In [80]:
testdf.ix[data[(data.target<0.7)&(data.target>0.29)].index]

,writing_id,body
19,57,「貴君《あなた》の作品の中《うち》で愛着を持つてゐらつしやるものか好きなものはありませんか」...
88,282,上何小二《かしょうじ》は軍刀を抛《ほう》り出すと夢中で馬の頸《くび》にしがみついた。確かに頸...
96,328,久米は官能の鋭敏な田舎者です。書くものばかりじゃありません。実生活上の趣味でも田舎者らしい所...
97,332,或春の午後であつた。私《わたし》は知人の田崎《たざき》に面会する為に彼が勤めてゐる出版｜書肆...
130,454,一天保二年九月の或午前である。神田同朋町《かんだどうぼうちやう》の銭湯《せんたう》松の湯では...
...,...,...
1359,4514,伴天連《ばてれん》うるがんの眼には外《ほか》の人の見えないものまでも見えたさうである。殊に人...
1361,4517,芸術家は何よりも作品の完成を期せねばならぬ。さもなければ芸術に奉仕する事が無意味になつてしま...
1363,4521,犬養君の作品は大抵読んでいるつもりである。その又僕の読んだ作品は何れも手を抜いたところはない...
1400,4663,広告この数篇の文章は何人かの人々を論じたものである。いやそれらの人々に対する僕の好悪《かうを...


In [49]:
data[(data.target<0.7)&(data.target>0.29)].target
#291 0 0.31
#412 1 0.61
#547 1 0.39 ✖︎
#625 1 0.34 ✖︎
#682 0 0.54 ✖︎
#702 1 0.56 
#774 0 0.37 
#815 1 0.35 ✖︎
#825 0 0.32 
#955 1 0.59
#962 0 0.29
#1029 1 0.42 ✖︎
#1050 0 0.29
#1116 1 0.60
#1140 1 0.31 ✖︎
#1194 1 0.65
#1200 0 0.42 
#1281 0 0.44 

291     0.311938
412     0.615388
547     0.394943
625     0.348613
682     0.540990
702     0.567220
774     0.376710
815     0.357455
825     0.321671
955     0.598116
962     0.290931
1029    0.428522
1050    0.290529
1116    0.609978
1140    0.315641
1194    0.653080
1200    0.428232
1281    0.442822
Name: target, dtype: float64

In [14]:
testdf.ix[data[(data.target<0.7)&(data.target>0.29)].index]

,writing_id,body
291,998,一私が生れて始めて蓄音器と云うものを見聞いたのはもう十四五年前父が英国から土産に買って来たも...
412,1379,昔々バグダツドのマホメツト教のお寺の前に一人の乞食が寝て居りました。丁度その時説教がすんだの...
547,1862,或る一つの作品を書かうと思つてそれが色々の径路を辿《たど》つてから出来上がる場合と直ぐ初めの...
625,2137,菊池は生き方が何時も徹底している。中途半端のところにこだわっていない。彼自身の正しいと思うと...
682,2321,十年前は阿佐ヶ谷に住んで居りやはり目下と同様吶々と小説ばかり書いて居りました。もう十年も経つ...
702,2385,すべて背景を用いない。宦官《かんがん》が二人話しながら出て来る。――今月も生み月になっている...
774,2649,ナポレオンの遺書――セント・ヘレナの島で臨終より三週間ほど前に彼が自ら口述し浄書したもので現...
815,2791,僕はこれからも今月のと同じような材料を使って創作するつもりである。あれを単なる歴史小説の仲間...
825,2828,一崇拝する偉人一現存の人の裡世界第一の偉人と思惟する人一右理由一プラトーシェイキスピアゲーテ...
955,3204,一ロマンスの中の女性は善悪共皆好み候。二あゝ云ふ女性は到底この世の中にゐないからに候。


In [71]:
testdf.body[1082]

'長二《ちょうじ》は貧乏《びんぼう》の家《いえ》に生《う》まれておもちゃも持《も》たずに死《し》んでしまった。美《うつく》しいガラス張《ば》りの店頭《みせさき》に西洋《せいよう》のぜいたくな小間物《こまもの》や赤《あか》紫《むらさき》に塗《ぬ》ったゴムまりやぴかぴかと顔《かお》の映《うつ》る銀笛《ぎんてき》やらっぱやなんでも子供《こども》の好《す》きそうなものが並《なら》べてあるのを見《み》ると店《みせ》のガラス戸《ど》を砕《くだ》いてそれらのものをめちゃめちゃにたたき壊《こわ》してやりたくなる。隣《となり》に住《す》んでいたあの貧《まず》しかった哀《あわ》れな長二《ちょうじ》のことを思《おも》い出《だ》したときに。'